# Importing Libraries

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time

# Extracting Payever Account page and its link

In [4]:
import os


# Provide the path to your ChromeDriver executable
chromedriver_path = 'C:/Drivers/chromedriver.exe'  

def init_driver():
    service = Service(executable_path=chromedriver_path)
    driver = webdriver.Chrome(service=service)
    return driver


driver = init_driver()

def extract_data_from_page(url):
    """Function to navigate to a specific page and extract its content."""
    try:
        driver.get(url)
        print(f"Loading page: {url}")
        
        # Increase timeout and use a more specific waiting condition
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.TAG_NAME, 'body'))
        )
        
        print(f"Successfully loaded: {url}")
        
        page_data = {'url': url}
        
        # Extract data from the specified tags
        try:
            page_data['header'] = driver.find_element(By.TAG_NAME, 'header').text
        except Exception:
            page_data['header'] = ''
            
        try:
            page_data['h1'] = driver.find_element(By.TAG_NAME, 'h1').text
        except Exception:
            page_data['h1'] = ''
        
        # Extracting all sections
        page_data['sections'] = []
        sections = driver.find_elements(By.TAG_NAME, 'section')
        for section in sections:
            section_data = {'h2': '', 'uls': []}
            
            try:
                section_data['h2'] = section.find_element(By.TAG_NAME, 'h2').text
            except Exception:
                section_data['h2'] = ''
            
            
            ul_elements = section.find_elements(By.TAG_NAME, 'ul')
            for ul in ul_elements:
                ul_data = {'text': ul.text, 'links': []}
                
                # Extract links from UL elements
                links = ul.find_elements(By.TAG_NAME, 'a')
                for link in links:
                    ul_data['links'].append(link.get_attribute('href'))
                
                section_data['uls'].append(ul_data)
            
            page_data['sections'].append(section_data)
        
        return page_data
    
    except Exception as e:
        print(f"Error loading page {url}: {e}")
        # Dump page source for debugging
        print(driver.page_source[:2000])  # Print first 2000 characters of the page source
        return {'url': url, 'header': '', 'h1': '', 'sections': []}

def extract_simple_data_from_page(url):
    """Function to navigate to a specific page and extract H1, P elements, and IMG tags."""
    global driver
    try:
        # Reinitialize the driver for each additional URL to avoid session issues
        driver.quit()
        time.sleep(2)  # Wait a moment before reinitializing
        driver = init_driver()

        driver.get(url)
        print(f"Loading additional page: {url}")
        
        # Increase timeout and use a more specific waiting condition
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.TAG_NAME, 'body'))
        )
        
        print(f"Successfully loaded: {url}")
        
        page_data = {'url': url}
        
        # Extract H1 element
        try:
            page_data['h1'] = driver.find_element(By.TAG_NAME, 'h1').text
        except Exception:
            page_data['h1'] = ''
        
        # Extract P elements
        page_data['paragraphs'] = []
        paragraphs = driver.find_elements(By.TAG_NAME, 'p')
        for p in paragraphs:
            page_data['paragraphs'].append(p.text)
        
        # Extract IMG elements
        page_data['images'] = []
        images = driver.find_elements(By.TAG_NAME, 'img')
        for img in images:
            src = img.get_attribute('src')
            alt = img.get_attribute('alt')
            page_data['images'].append({'src': src, 'alt': alt})
        # Extract VIDEO elements
        page_data['videos'] = []
        videos = driver.find_elements(By.TAG_NAME, 'video')
        for video in videos:
            video_data = {
                'src': video.get_attribute('src'),
                'controls': video.get_attribute('controls'),
                'autoplay': video.get_attribute('autoplay'),
                'loop': video.get_attribute('loop'),
                'muted': video.get_attribute('muted')
            }
            page_data['videos'].append(video_data)
        
        return page_data
    
    except Exception as e:
        print(f"Error loading page {url}: {e}")
        # Dump page source for debugging
        print(driver.page_source[:2000])  # Print first 2000 characters of the page source
        return {'url': url, 'h1': '', 'paragraphs': [], 'images': [], 'videos': []}


main_url = 'https://support.payever.org/hc/en-us/categories/360002505659-payever-Account'


main_page_data = extract_data_from_page(main_url)


additional_data = []
for section in main_page_data['sections']:
    for ul in section['uls']:
        additional_urls = ul['links']
        for additional_url in additional_urls:
            additional_page_data = extract_simple_data_from_page(additional_url)
            additional_data.append(additional_page_data)

main_page_data['additional_data'] = additional_data


with open('scraped_data.json', 'w') as f:
    json.dump(main_page_data, f, indent=4)

# Print the data
print(f"URL: {main_page_data['url']}")
print(f"Header: {main_page_data['header']}")
print(f"H1: {main_page_data['h1']}")
print("Sections:")
for section in main_page_data['sections']:
    print(f"  H2: {section['h2']}")
    for ul in section['uls']:
        print(f"    UL: {ul['text']}")
        print(f"    Links: {ul['links']}")
        for additional in additional_data:
            if additional['url'] in ul['links']:
                print(f"      Additional URL: {additional['url']}")
                print(f"      Additional H1: {additional['h1']}")
                print(f"      Paragraphs: {additional['paragraphs']}")
                print(f"      Images: {additional['images']}")
                print(f"      Images: {additional['videos']}")
    print("\n" + "="*40 + "\n")


driver.quit()


Loading page: https://support.payever.org/hc/en-us/categories/360002505659-payever-Account
Successfully loaded: https://support.payever.org/hc/en-us/categories/360002505659-payever-Account
Loading additional page: https://support.payever.org/hc/en-us/articles/360013450500-Navigate-through-the-payever-dashboard
Successfully loaded: https://support.payever.org/hc/en-us/articles/360013450500-Navigate-through-the-payever-dashboard
Loading additional page: https://support.payever.org/hc/en-us/articles/9745946743708-Basic-procedure-Guidance-for-employees
Successfully loaded: https://support.payever.org/hc/en-us/articles/9745946743708-Basic-procedure-Guidance-for-employees
Loading additional page: https://support.payever.org/hc/en-us/articles/6699730902044-Adding-multiple-businesses
Successfully loaded: https://support.payever.org/hc/en-us/articles/6699730902044-Adding-multiple-businesses
Loading additional page: https://support.payever.org/hc/en-us/articles/360013623379-Apps-and-Widget-confi

# Extracting payever-Settings page and it's link

In [36]:
import os

driver = init_driver()

main_url = 'https://support.payever.org/hc/en-us/categories/360002490100-payever-Settings'


main_page_data = extract_data_from_page(main_url)


additional_data = []
for section in main_page_data['sections']:
    for ul in section['uls']:
        additional_urls = ul['links']
        for additional_url in additional_urls:
            additional_page_data = extract_simple_data_from_page(additional_url)
            additional_data.append(additional_page_data)



main_page_data['additional_data'] = additional_data


if os.path.exists('scraped_data.json'):
    print('It is available')
    with open('scraped_data.json', 'r') as f:
        existing_data = json.load(f)
        
    # Extend the existing additional_data with the new additional_data
    if 'additional_data' in existing_data:
        existing_data['additional_data'].extend(additional_data)
    else:
        existing_data['additional_data'] = additional_data

    # Convert the dictionary to a JSON string
    json_string = json.dumps(existing_data, indent=4, separators=(',', ': '))

    # Insert two line breaks between JSON entries
    formatted_json_string = json_string.replace('\n', '\n\n')
    
    # Write the formatted JSON string back to the file
    with open('scraped_data.json', 'w') as f:
        f.write(formatted_json_string)
else:
    # Save data if file does not exist
    main_page_data['additional_data'] = additional_data
    with open('scraped_data.json', 'w') as f:
        json.dump(main_page_data, f, indent=4, separators=(',', ': '))
# Print the data
print(f"URL: {main_page_data['url']}")
print(f"Header: {main_page_data['header']}")
print(f"H1: {main_page_data['h1']}")
print("Sections:")
for section in main_page_data['sections']:
    print(f"  H2: {section['h2']}")
    for ul in section['uls']:
        print(f"    UL: {ul['text']}")
        print(f"    Links: {ul['links']}")
        for additional in additional_data:
            if additional['url'] in ul['links']:
                print(f"      Additional URL: {additional['url']}")
                print(f"      Additional H1: {additional['h1']}")
                print(f"      Paragraphs: {additional['paragraphs']}")
                print(f"      Images: {additional['images']}")
    print("\n" + "="*40 + "\n")


driver.quit()

Loading page: https://support.payever.org/hc/en-us/categories/360002490100-payever-Settings
Successfully loaded: https://support.payever.org/hc/en-us/categories/360002490100-payever-Settings
Loading additional page: https://support.payever.org/hc/en-us/articles/360013519619-Set-up-payever-Settings
Successfully loaded: https://support.payever.org/hc/en-us/articles/360013519619-Set-up-payever-Settings
Loading additional page: https://support.payever.org/hc/en-us/articles/360013574880-Employee-management
Successfully loaded: https://support.payever.org/hc/en-us/articles/360013574880-Employee-management
Loading additional page: https://support.payever.org/hc/en-us/articles/360013625339-Customize-the-wallpaper
Successfully loaded: https://support.payever.org/hc/en-us/articles/360013625339-Customize-the-wallpaper
It is available
URL: https://support.payever.org/hc/en-us/categories/360002490100-payever-Settings
Header: Sign in
H1: payever Settings
Sections:
  H2: Getting Started
    UL: Set u

# Extracting payever-Checkout page and it's link

In [35]:
# Initialize the driver
driver = init_driver()

main_url = 'https://support.payever.org/hc/en-us/sections/360004467653-Getting-started'


main_page_data = extract_data_from_page(main_url)

# Extract additional data from links within the UL elements
additional_data = []
for section in main_page_data['sections']:
    for ul in section['uls']:
        additional_urls = ul['links']
        for additional_url in additional_urls:
            additional_page_data = extract_simple_data_from_page(additional_url)
            additional_data.append(additional_page_data)




main_page_data['additional_data'] = additional_data

# Check if the file exists
if os.path.exists('scraped_data.json'):
    print('It is available')
    with open('scraped_data.json', 'r') as f:
        existing_data = json.load(f)
        
    # Extend the existing additional_data with the new additional_data
    if 'additional_data' in existing_data:
        existing_data['additional_data'].extend(additional_data)
    else:
        existing_data['additional_data'] = additional_data

    
    json_string = json.dumps(existing_data, indent=4, separators=(',', ': '))

    # Insert two line breaks between JSON entries
    formatted_json_string = json_string.replace('\n', '\n\n')
    
    # Write the formatted JSON string back to the file
    with open('scraped_data.json', 'w') as f:
        f.write(formatted_json_string)
else:
    # Save data if file does not exist
    main_page_data['additional_data'] = additional_data
    with open('scraped_data.json', 'w') as f:
        json.dump(main_page_data, f, indent=4, separators=(',', ': '))

# Print the data
print(f"URL: {main_page_data['url']}")
print(f"Header: {main_page_data['header']}")
print(f"H1: {main_page_data['h1']}")
print("Sections:")
for section in main_page_data['sections']:
    print(f"  H2: {section['h2']}")
    for ul in section['uls']:
        print(f"    UL: {ul['text']}")
        print(f"    Links: {ul['links']}")
        for additional in additional_data:
            if additional['url'] in ul['links']:
                print(f"      Additional URL: {additional['url']}")
                print(f"      Additional H1: {additional['h1']}")
                print(f"      Paragraphs: {additional['paragraphs']}")
                print(f"      Images: {additional['images']}")
    print("\n" + "="*40 + "\n")
driver.quit()

Loading page: https://support.payever.org/hc/en-us/sections/360004467653-Getting-started
Successfully loaded: https://support.payever.org/hc/en-us/sections/360004467653-Getting-started
Loading additional page: https://support.payever.org/hc/en-us/articles/360009828420-Create-an-account-in-payever
Successfully loaded: https://support.payever.org/hc/en-us/articles/360009828420-Create-an-account-in-payever
Loading additional page: https://support.payever.org/hc/en-us/articles/360013328080-Navigate-through-the-payever-dashboard
Successfully loaded: https://support.payever.org/hc/en-us/articles/360013328080-Navigate-through-the-payever-dashboard
Loading additional page: https://support.payever.org/hc/en-us/articles/360023893874-Set-up-a-checkout
Successfully loaded: https://support.payever.org/hc/en-us/articles/360023893874-Set-up-a-checkout
Loading additional page: https://support.payever.org/hc/en-us/articles/360012729660-Employee-management
Successfully loaded: https://support.payever.or

# Extracting payever-Checkout page and it's link

In [38]:
driver = init_driver()

main_url = 'https://support.payever.org/hc/en-us/sections/360004467673-Payment-Options'

# Collect data from the main page
main_page_data = extract_data_from_page(main_url)

# Extract additional data from links within the UL elements
additional_data = []
for section in main_page_data['sections']:
    for ul in section['uls']:
        additional_urls = ul['links']
        for additional_url in additional_urls:
            additional_page_data = extract_simple_data_from_page(additional_url)
            additional_data.append(additional_page_data)



# Add additional data to the main page data
main_page_data['additional_data'] = additional_data

# Check if the file exists
if os.path.exists('scraped_data.json'):
    print('It is available')
    with open('scraped_data.json', 'r') as f:
        existing_data = json.load(f)
        
    # Extend the existing additional_data with the new additional_data
    if 'additional_data' in existing_data:
        existing_data['additional_data'].extend(additional_data)
    else:
        existing_data['additional_data'] = additional_data

    # Convert the dictionary to a JSON string
    json_string = json.dumps(existing_data, indent=4, separators=(',', ': '))

    # Insert two line breaks between JSON entries
    formatted_json_string = json_string.replace('\n', '\n\n')
    
    # Write the formatted JSON string back to the file
    with open('scraped_data.json', 'w') as f:
        f.write(formatted_json_string)
else:
    # Save data if file does not exist
    main_page_data['additional_data'] = additional_data
    with open('scraped_data.json', 'w') as f:
        json.dump(main_page_data, f, indent=4, separators=(',', ': '))

# Print the data
print(f"URL: {main_page_data['url']}")
print(f"Header: {main_page_data['header']}")
print(f"H1: {main_page_data['h1']}")
print("Sections:")
for section in main_page_data['sections']:
    print(f"  H2: {section['h2']}")
    for ul in section['uls']:
        print(f"    UL: {ul['text']}")
        print(f"    Links: {ul['links']}")
        for additional in additional_data:
            if additional['url'] in ul['links']:
                print(f"      Additional URL: {additional['url']}")
                print(f"      Additional H1: {additional['h1']}")
                print(f"      Paragraphs: {additional['paragraphs']}")
                print(f"      Images: {additional['images']}")
    print("\n" + "="*40 + "\n")


# Quit the driver at the end
driver.quit()

Loading page: https://support.payever.org/hc/en-us/sections/360004467673-Payment-Options
Successfully loaded: https://support.payever.org/hc/en-us/sections/360004467673-Payment-Options
Loading additional page: https://support.payever.org/hc/en-us/articles/6024376250140-Zinia-by-Santander-Enjoy-Now-Pay-Later-Germany-Netherlands-Spain
Successfully loaded: https://support.payever.org/hc/en-us/articles/6024376250140-Zinia-by-Santander-Enjoy-Now-Pay-Later-Germany-Netherlands-Spain
Loading additional page: https://support.payever.org/hc/en-us/articles/13564976628508-Allianz-Trade-pay-B2B-Buy-Now-Pay-Later-Worldwide
Successfully loaded: https://support.payever.org/hc/en-us/articles/13564976628508-Allianz-Trade-pay-B2B-Buy-Now-Pay-Later-Worldwide
Loading additional page: https://support.payever.org/hc/en-us/articles/360023899574-Santander-Installments-Financing-Germany
Successfully loaded: https://support.payever.org/hc/en-us/articles/360023899574-Santander-Installments-Financing-Germany
Loadi

# Extracting payever-Checkout page and it's link

In [40]:
driver = init_driver()


main_url = 'https://support.payever.org/hc/en-us/sections/360004418894-Plugin-Integrations'

# Collect data from the main page
main_page_data = extract_data_from_page(main_url)

# Extract additional data from links within the UL elements
additional_data = []
for section in main_page_data['sections']:
    for ul in section['uls']:
        additional_urls = ul['links']
        for additional_url in additional_urls:
            additional_page_data = extract_simple_data_from_page(additional_url)
            additional_data.append(additional_page_data)



# Add additional data to the main page data
main_page_data['additional_data'] = additional_data

# Check if the file exists
if os.path.exists('scraped_data.json'):
    print('It is available')
    with open('scraped_data.json', 'r') as f:
        existing_data = json.load(f)
        
    # Extend the existing additional_data with the new additional_data
    if 'additional_data' in existing_data:
        existing_data['additional_data'].extend(additional_data)
    else:
        existing_data['additional_data'] = additional_data

    # Convert the dictionary to a JSON string
    json_string = json.dumps(existing_data, indent=4, separators=(',', ': '))

    # Insert two line breaks between JSON entries
    formatted_json_string = json_string.replace('\n', '\n\n')
    
    # Write the formatted JSON string back to the file
    with open('scraped_data.json', 'w') as f:
        f.write(formatted_json_string)
else:
    # Save data if file does not exist
    main_page_data['additional_data'] = additional_data
    with open('scraped_data.json', 'w') as f:
        json.dump(main_page_data, f, indent=4, separators=(',', ': '))

# Print the data
print(f"URL: {main_page_data['url']}")
print(f"Header: {main_page_data['header']}")
print(f"H1: {main_page_data['h1']}")
print("Sections:")
for section in main_page_data['sections']:
    print(f"  H2: {section['h2']}")
    for ul in section['uls']:
        print(f"    UL: {ul['text']}")
        print(f"    Links: {ul['links']}")
        for additional in additional_data:
            if additional['url'] in ul['links']:
                print(f"      Additional URL: {additional['url']}")
                print(f"      Additional H1: {additional['h1']}")
                print(f"      Paragraphs: {additional['paragraphs']}")
                print(f"      Images: {additional['images']}")
    print("\n" + "="*40 + "\n")


# Quit the driver at the end
driver.quit()

Loading page: https://support.payever.org/hc/en-us/sections/360004418894-Plugin-Integrations
Successfully loaded: https://support.payever.org/hc/en-us/sections/360004418894-Plugin-Integrations
Loading additional page: https://support.payever.org/hc/en-us/articles/360011228539-Test-Data
Successfully loaded: https://support.payever.org/hc/en-us/articles/360011228539-Test-Data
Loading additional page: https://support.payever.org/hc/en-us/articles/360020713660-payever-Plugin-License-Copyright-and-Intellectual-Property
Successfully loaded: https://support.payever.org/hc/en-us/articles/360020713660-payever-Plugin-License-Copyright-and-Intellectual-Property
Loading additional page: https://support.payever.org/hc/en-us/articles/360012435820-Shopware-6
Successfully loaded: https://support.payever.org/hc/en-us/articles/360012435820-Shopware-6
Loading additional page: https://support.payever.org/hc/en-us/articles/360023894874-Shopware-5
Successfully loaded: https://support.payever.org/hc/en-us/ar

# Extracting payever-Checkout page and it's link

In [41]:
driver = init_driver()


main_url = 'https://support.payever.org/hc/en-us/sections/360004467573-Conditions'

# Collect data from the main page
main_page_data = extract_data_from_page(main_url)

# Extract additional data from links within the UL elements
additional_data = []
for section in main_page_data['sections']:
    for ul in section['uls']:
        additional_urls = ul['links']
        for additional_url in additional_urls:
            additional_page_data = extract_simple_data_from_page(additional_url)
            additional_data.append(additional_page_data)



# Add additional data to the main page data
main_page_data['additional_data'] = additional_data

# Check if the file exists
if os.path.exists('scraped_data.json'):
    print('It is available')
    with open('scraped_data.json', 'r') as f:
        existing_data = json.load(f)
        
    # Extend the existing additional_data with the new additional_data
    if 'additional_data' in existing_data:
        existing_data['additional_data'].extend(additional_data)
    else:
        existing_data['additional_data'] = additional_data

    # Convert the dictionary to a JSON string
    json_string = json.dumps(existing_data, indent=4, separators=(',', ': '))

    # Insert two line breaks between JSON entries
    formatted_json_string = json_string.replace('\n', '\n\n')
    
    # Write the formatted JSON string back to the file
    with open('scraped_data.json', 'w') as f:
        f.write(formatted_json_string)
else:
    # Save data if file does not exist
    main_page_data['additional_data'] = additional_data
    with open('scraped_data.json', 'w') as f:
        json.dump(main_page_data, f, indent=4, separators=(',', ': '))

# Print the data
print(f"URL: {main_page_data['url']}")
print(f"Header: {main_page_data['header']}")
print(f"H1: {main_page_data['h1']}")
print("Sections:")
for section in main_page_data['sections']:
    print(f"  H2: {section['h2']}")
    for ul in section['uls']:
        print(f"    UL: {ul['text']}")
        print(f"    Links: {ul['links']}")
        for additional in additional_data:
            if additional['url'] in ul['links']:
                print(f"      Additional URL: {additional['url']}")
                print(f"      Additional H1: {additional['h1']}")
                print(f"      Paragraphs: {additional['paragraphs']}")
                print(f"      Images: {additional['images']}")
    print("\n" + "="*40 + "\n")


# Quit the driver at the end
driver.quit()

Loading page: https://support.payever.org/hc/en-us/sections/360004467573-Conditions
Successfully loaded: https://support.payever.org/hc/en-us/sections/360004467573-Conditions
Loading additional page: https://support.payever.org/hc/en-us/articles/360024065693-Availability-of-payment-options
Successfully loaded: https://support.payever.org/hc/en-us/articles/360024065693-Availability-of-payment-options
Loading additional page: https://support.payever.org/hc/en-us/articles/360024064313-Min-and-max-amounts-of-a-transaction
Successfully loaded: https://support.payever.org/hc/en-us/articles/360024064313-Min-and-max-amounts-of-a-transaction
Loading additional page: https://support.payever.org/hc/en-us/articles/360023893454-Transaction-fees
Successfully loaded: https://support.payever.org/hc/en-us/articles/360023893454-Transaction-fees
It is available
URL: https://support.payever.org/hc/en-us/sections/360004467573-Conditions
Header: Sign in
H1: Conditions
Sections:
  H2: 
    UL: Availability o

# Extracting payever-Checkout page and it's link

In [42]:
driver = init_driver()

main_url = 'https://support.payever.org/hc/en-us/sections/360003015860-Pay-by-Link'

# Collect data from the main page
main_page_data = extract_data_from_page(main_url)

# Extract additional data from links within the UL elements
additional_data = []
for section in main_page_data['sections']:
    for ul in section['uls']:
        additional_urls = ul['links']
        for additional_url in additional_urls:
            additional_page_data = extract_simple_data_from_page(additional_url)
            additional_data.append(additional_page_data)



# Add additional data to the main page data
main_page_data['additional_data'] = additional_data

# Check if the file exists
if os.path.exists('scraped_data.json'):
    print('It is available')
    with open('scraped_data.json', 'r') as f:
        existing_data = json.load(f)
        
    # Extend the existing additional_data with the new additional_data
    if 'additional_data' in existing_data:
        existing_data['additional_data'].extend(additional_data)
    else:
        existing_data['additional_data'] = additional_data

    # Convert the dictionary to a JSON string
    json_string = json.dumps(existing_data, indent=4, separators=(',', ': '))

    # Insert two line breaks between JSON entries
    formatted_json_string = json_string.replace('\n', '\n\n')
    
    # Write the formatted JSON string back to the file
    with open('scraped_data.json', 'w') as f:
        f.write(formatted_json_string)
else:
    # Save data if file does not exist
    main_page_data['additional_data'] = additional_data
    with open('scraped_data.json', 'w') as f:
        json.dump(main_page_data, f, indent=4, separators=(',', ': '))

# Print the data
print(f"URL: {main_page_data['url']}")
print(f"Header: {main_page_data['header']}")
print(f"H1: {main_page_data['h1']}")
print("Sections:")
for section in main_page_data['sections']:
    print(f"  H2: {section['h2']}")
    for ul in section['uls']:
        print(f"    UL: {ul['text']}")
        print(f"    Links: {ul['links']}")
        for additional in additional_data:
            if additional['url'] in ul['links']:
                print(f"      Additional URL: {additional['url']}")
                print(f"      Additional H1: {additional['h1']}")
                print(f"      Paragraphs: {additional['paragraphs']}")
                print(f"      Images: {additional['images']}")
    print("\n" + "="*40 + "\n")


# Quit the driver at the end
driver.quit()

Loading page: https://support.payever.org/hc/en-us/sections/360003015860-Pay-by-Link
Successfully loaded: https://support.payever.org/hc/en-us/sections/360003015860-Pay-by-Link
Loading additional page: https://support.payever.org/hc/en-us/articles/360013539020-payever-Checkout-Pay-by-Link-Quick-Guide
Successfully loaded: https://support.payever.org/hc/en-us/articles/360013539020-payever-Checkout-Pay-by-Link-Quick-Guide
Loading additional page: https://support.payever.org/hc/en-us/articles/360013580079-payever-Checkout-Pay-by-Link-Detailed-Instruction
Successfully loaded: https://support.payever.org/hc/en-us/articles/360013580079-payever-Checkout-Pay-by-Link-Detailed-Instruction
It is available
URL: https://support.payever.org/hc/en-us/sections/360003015860-Pay-by-Link
Header: Sign in
H1: Pay by Link
Sections:
  H2: 
    UL: payever Checkout - Pay by Link (Quick Guide)
payever Checkout - Pay by Link (Detailed Instruction)
    Links: ['https://support.payever.org/hc/en-us/articles/360013

# Extracting payever-Checkout page and it's link

In [44]:
driver = init_driver()

# Specify the main URL to scrape
main_url = 'https://support.payever.org/hc/en-us/sections/360003088179-Widgets'

# Collect data from the main page
main_page_data = extract_data_from_page(main_url)

# Extract additional data from links within the UL elements
additional_data = []
for section in main_page_data['sections']:
    for ul in section['uls']:
        additional_urls = ul['links']
        for additional_url in additional_urls:
            additional_page_data = extract_simple_data_from_page(additional_url)
            additional_data.append(additional_page_data)



# Add additional data to the main page data
main_page_data['additional_data'] = additional_data

# Check if the file exists
if os.path.exists('scraped_data.json'):
    print('It is available')
    with open('scraped_data.json', 'r') as f:
        existing_data = json.load(f)
        
    # Extend the existing additional_data with the new additional_data
    if 'additional_data' in existing_data:
        existing_data['additional_data'].extend(additional_data)
    else:
        existing_data['additional_data'] = additional_data

    # Convert the dictionary to a JSON string
    json_string = json.dumps(existing_data, indent=4, separators=(',', ': '))

    # Insert two line breaks between JSON entries
    formatted_json_string = json_string.replace('\n', '\n\n')
    
    # Write the formatted JSON string back to the file
    with open('scraped_data.json', 'w') as f:
        f.write(formatted_json_string)
else:
    # Save data if file does not exist
    main_page_data['additional_data'] = additional_data
    with open('scraped_data.json', 'w') as f:
        json.dump(main_page_data, f, indent=4, separators=(',', ': '))

# Print the data
print(f"URL: {main_page_data['url']}")
print(f"Header: {main_page_data['header']}")
print(f"H1: {main_page_data['h1']}")
print("Sections:")
for section in main_page_data['sections']:
    print(f"  H2: {section['h2']}")
    for ul in section['uls']:
        print(f"    UL: {ul['text']}")
        print(f"    Links: {ul['links']}")
        for additional in additional_data:
            if additional['url'] in ul['links']:
                print(f"      Additional URL: {additional['url']}")
                print(f"      Additional H1: {additional['h1']}")
                print(f"      Paragraphs: {additional['paragraphs']}")
                print(f"      Images: {additional['images']}")
    print("\n" + "="*40 + "\n")


# Quit the driver at the end
driver.quit()

Loading page: https://support.payever.org/hc/en-us/sections/360003088179-Widgets
Successfully loaded: https://support.payever.org/hc/en-us/sections/360003088179-Widgets
Loading additional page: https://support.payever.org/hc/en-us/articles/11879035290396-Implement-Express-Checkout
Successfully loaded: https://support.payever.org/hc/en-us/articles/11879035290396-Implement-Express-Checkout
Loading additional page: https://support.payever.org/hc/en-us/articles/9578013738396-WooCommerce-Ivy-Widget
Successfully loaded: https://support.payever.org/hc/en-us/articles/9578013738396-WooCommerce-Ivy-Widget
Loading additional page: https://support.payever.org/hc/en-us/articles/360010325400-Implement-widgets
Successfully loaded: https://support.payever.org/hc/en-us/articles/360010325400-Implement-widgets
It is available
URL: https://support.payever.org/hc/en-us/sections/360003088179-Widgets
Header: Sign in
H1: Widgets
Sections:
  H2: 
    UL: Implement Express Checkout
WooCommerce Ivy Widget
Implem

# Extracting payever-Point-of-Sale page and it's link

In [48]:
driver = init_driver()

main_url = 'https://support.payever.org/hc/en-us/categories/360001887713-payever-Point-of-Sale'

# Collect data from the main page
main_page_data = extract_data_from_page(main_url)

# Extract additional data from links within the UL elements
additional_data = []
for section in main_page_data['sections']:
    for ul in section['uls']:
        additional_urls = ul['links']
        for additional_url in additional_urls:
            additional_page_data = extract_simple_data_from_page(additional_url)
            additional_data.append(additional_page_data)



# Add additional data to the main page data
main_page_data['additional_data'] = additional_data

# Check if the file exists
if os.path.exists('scraped_data.json'):
    print('It is available')
    with open('scraped_data.json', 'r') as f:
        existing_data = json.load(f)
        
    # Extend the existing additional_data with the new additional_data
    if 'additional_data' in existing_data:
        existing_data['additional_data'].extend(additional_data)
    else:
        existing_data['additional_data'] = additional_data

    # Convert the dictionary to a JSON string
    json_string = json.dumps(existing_data, indent=4, separators=(',', ': '))

    # Insert two line breaks between JSON entries
    formatted_json_string = json_string.replace('\n', '\n\n')
    
    # Write the formatted JSON string back to the file
    with open('scraped_data.json', 'w') as f:
        f.write(formatted_json_string)
else:
    # Save data if file does not exist
    main_page_data['additional_data'] = additional_data
    with open('scraped_data.json', 'w') as f:
        json.dump(main_page_data, f, indent=4, separators=(',', ': '))

# Print the data
print(f"URL: {main_page_data['url']}")
print(f"Header: {main_page_data['header']}")
print(f"H1: {main_page_data['h1']}")
print("Sections:")
for section in main_page_data['sections']:
    print(f"  H2: {section['h2']}")
    for ul in section['uls']:
        print(f"    UL: {ul['text']}")
        print(f"    Links: {ul['links']}")
        for additional in additional_data:
            if additional['url'] in ul['links']:
                print(f"      Additional URL: {additional['url']}")
                print(f"      Additional H1: {additional['h1']}")
                print(f"      Paragraphs: {additional['paragraphs']}")
                print(f"      Images: {additional['images']}")
    print("\n" + "="*40 + "\n")


# Quit the driver at the end
driver.quit()

Loading page: https://support.payever.org/hc/en-us/categories/360001887713-payever-Point-of-Sale
Successfully loaded: https://support.payever.org/hc/en-us/categories/360001887713-payever-Point-of-Sale
Loading additional page: https://support.payever.org/hc/en-us/articles/360013579799-Setting-up-your-PoS-terminal
Successfully loaded: https://support.payever.org/hc/en-us/articles/360013579799-Setting-up-your-PoS-terminal
Loading additional page: https://support.payever.org/hc/en-us/articles/360013582140-Employee-management
Successfully loaded: https://support.payever.org/hc/en-us/articles/360013582140-Employee-management
Loading additional page: https://support.payever.org/hc/en-us/articles/6243732210332-Generate-QR-Code
Successfully loaded: https://support.payever.org/hc/en-us/articles/6243732210332-Generate-QR-Code
Loading additional page: https://support.payever.org/hc/en-us/articles/9213316052764-Zinia-Buy-Now-Pay-Later-Germany
Successfully loaded: https://support.payever.org/hc/en-u

# Extracting payever-Connect page and it's link

In [58]:
# Initialize the driver
driver = init_driver()

# Specify the main URL to scrape
main_url = 'https://support.payever.org/hc/en-us/categories/360002505599-payever-Connect'

# Collect data from the main page
main_page_data = extract_data_from_page(main_url)

# Extract additional data from links within the UL elements
additional_data = []
for section in main_page_data['sections']:
    for ul in section['uls']:
        additional_urls = ul['links']
        for additional_url in additional_urls:
            additional_page_data = extract_simple_data_from_page(additional_url)
            additional_data.append(additional_page_data)



# Add additional data to the main page data
main_page_data['additional_data'] = additional_data

# Check if the file exists
if os.path.exists('scraped_data.json'):
    print('It is available')
    with open('scraped_data.json', 'r') as f:
        existing_data = json.load(f)
        
    # Extend the existing additional_data with the new additional_data
    if 'additional_data' in existing_data:
        existing_data['additional_data'].extend(additional_data)
    else:
        existing_data['additional_data'] = additional_data

    # Convert the dictionary to a JSON string
    json_string = json.dumps(existing_data, indent=4, separators=(',', ': '))

    # Insert two line breaks between JSON entries
    formatted_json_string = json_string.replace('\n', '\n\n')
    
    # Write the formatted JSON string back to the file
    with open('scraped_data.json', 'w') as f:
        f.write(formatted_json_string)
else:
    # Save data if file does not exist
    main_page_data['additional_data'] = additional_data
    with open('scraped_data.json', 'w') as f:
        json.dump(main_page_data, f, indent=4, separators=(',', ': '))

# Print the data
print(f"URL: {main_page_data['url']}")
print(f"Header: {main_page_data['header']}")
print(f"H1: {main_page_data['h1']}")
print("Sections:")
for section in main_page_data['sections']:
    print(f"  H2: {section['h2']}")
    for ul in section['uls']:
        print(f"    UL: {ul['text']}")
        print(f"    Links: {ul['links']}")
        for additional in additional_data:
            if additional['url'] in ul['links']:
                print(f"      Additional URL: {additional['url']}")
                print(f"      Additional H1: {additional['h1']}")
                print(f"      Paragraphs: {additional['paragraphs']}")
                print(f"      Images: {additional['images']}")
    print("\n" + "="*40 + "\n")


# Quit the driver at the end
driver.quit()

Loading page: https://support.payever.org/hc/en-us/categories/360002505599-payever-Connect
Successfully loaded: https://support.payever.org/hc/en-us/categories/360002505599-payever-Connect
Loading additional page: https://support.payever.org/hc/en-us/articles/360013449980-Set-up-payever-Connect
Successfully loaded: https://support.payever.org/hc/en-us/articles/360013449980-Set-up-payever-Connect
Loading additional page: https://support.payever.org/hc/en-us/articles/9840455246748-Configure-communication-settings
Successfully loaded: https://support.payever.org/hc/en-us/articles/9840455246748-Configure-communication-settings
Loading additional page: https://support.payever.org/hc/en-us/articles/9872594703388-Configure-social-media
Successfully loaded: https://support.payever.org/hc/en-us/articles/9872594703388-Configure-social-media
Loading additional page: https://support.payever.org/hc/en-us/articles/9868662089628-Configure-shipping-options
Successfully loaded: https://support.payever.

# Extracting payever-Products page and it's link

In [51]:
# Initialize the driver
driver = init_driver()


# Specify the main URL to scrape
main_url = 'https://support.payever.org/hc/en-us/categories/360001870634-payever-Products'

# Collect data from the main page
main_page_data = extract_data_from_page(main_url)

# Extract additional data from links within the UL elements
additional_data = []
for section in main_page_data['sections']:
    for ul in section['uls']:
        additional_urls = ul['links']
        for additional_url in additional_urls:
            additional_page_data = extract_simple_data_from_page(additional_url)
            additional_data.append(additional_page_data)



# Add additional data to the main page data
main_page_data['additional_data'] = additional_data

# Check if the file exists
if os.path.exists('scraped_data.json'):
    print('It is available')
    with open('scraped_data.json', 'r') as f:
        existing_data = json.load(f)
        
    # Extend the existing additional_data with the new additional_data
    if 'additional_data' in existing_data:
        existing_data['additional_data'].extend(additional_data)
    else:
        existing_data['additional_data'] = additional_data

    # Convert the dictionary to a JSON string
    json_string = json.dumps(existing_data, indent=4, separators=(',', ': '))

    # Insert two line breaks between JSON entries
    formatted_json_string = json_string.replace('\n', '\n\n')
    
    # Write the formatted JSON string back to the file
    with open('scraped_data.json', 'w') as f:
        f.write(formatted_json_string)
else:
    # Save data if file does not exist
    main_page_data['additional_data'] = additional_data
    with open('scraped_data.json', 'w') as f:
        json.dump(main_page_data, f, indent=4, separators=(',', ': '))

# Print the data
print(f"URL: {main_page_data['url']}")
print(f"Header: {main_page_data['header']}")
print(f"H1: {main_page_data['h1']}")
print("Sections:")
for section in main_page_data['sections']:
    print(f"  H2: {section['h2']}")
    for ul in section['uls']:
        print(f"    UL: {ul['text']}")
        print(f"    Links: {ul['links']}")
        for additional in additional_data:
            if additional['url'] in ul['links']:
                print(f"      Additional URL: {additional['url']}")
                print(f"      Additional H1: {additional['h1']}")
                print(f"      Paragraphs: {additional['paragraphs']}")
                print(f"      Images: {additional['images']}")
    print("\n" + "="*40 + "\n")


# Quit the driver at the end
driver.quit()

Loading page: https://support.payever.org/hc/en-us/categories/360001870634-payever-Products
Successfully loaded: https://support.payever.org/hc/en-us/categories/360001870634-payever-Products
Loading additional page: https://support.payever.org/hc/en-us/articles/4663758443794-Instagram-Shopping-Create-and-import-products-in-payever-and-connect-to-Instagram
Successfully loaded: https://support.payever.org/hc/en-us/articles/4663758443794-Instagram-Shopping-Create-and-import-products-in-payever-and-connect-to-Instagram
Loading additional page: https://support.payever.org/hc/en-us/articles/360024064893-Create-products-in-the-payever-Products-App
Successfully loaded: https://support.payever.org/hc/en-us/articles/360024064893-Create-products-in-the-payever-Products-App
Loading additional page: https://support.payever.org/hc/en-us/articles/4402863103762-Import-your-products-to-payever-via-CSV
Successfully loaded: https://support.payever.org/hc/en-us/articles/4402863103762-Import-your-products-

# Extracting payever-Shop page and it's link

In [52]:
# Initialize the driver
driver = init_driver()

# Specify the main URL to scrape
main_url = 'https://support.payever.org/hc/en-us/categories/360001870554-payever-Shop'

# Collect data from the main page
main_page_data = extract_data_from_page(main_url)

# Extract additional data from links within the UL elements
additional_data = []
for section in main_page_data['sections']:
    for ul in section['uls']:
        additional_urls = ul['links']
        for additional_url in additional_urls:
            additional_page_data = extract_simple_data_from_page(additional_url)
            additional_data.append(additional_page_data)



# Add additional data to the main page data
main_page_data['additional_data'] = additional_data

# Check if the file exists
if os.path.exists('scraped_data.json'):
    print('It is available')
    with open('scraped_data.json', 'r') as f:
        existing_data = json.load(f)
        
    # Extend the existing additional_data with the new additional_data
    if 'additional_data' in existing_data:
        existing_data['additional_data'].extend(additional_data)
    else:
        existing_data['additional_data'] = additional_data

    # Convert the dictionary to a JSON string
    json_string = json.dumps(existing_data, indent=4, separators=(',', ': '))

    # Insert two line breaks between JSON entries
    formatted_json_string = json_string.replace('\n', '\n\n')
    
    # Write the formatted JSON string back to the file
    with open('scraped_data.json', 'w') as f:
        f.write(formatted_json_string)
else:
    # Save data if file does not exist
    main_page_data['additional_data'] = additional_data
    with open('scraped_data.json', 'w') as f:
        json.dump(main_page_data, f, indent=4, separators=(',', ': '))

# Print the data
print(f"URL: {main_page_data['url']}")
print(f"Header: {main_page_data['header']}")
print(f"H1: {main_page_data['h1']}")
print("Sections:")
for section in main_page_data['sections']:
    print(f"  H2: {section['h2']}")
    for ul in section['uls']:
        print(f"    UL: {ul['text']}")
        print(f"    Links: {ul['links']}")
        for additional in additional_data:
            if additional['url'] in ul['links']:
                print(f"      Additional URL: {additional['url']}")
                print(f"      Additional H1: {additional['h1']}")
                print(f"      Paragraphs: {additional['paragraphs']}")
                print(f"      Images: {additional['images']}")
    print("\n" + "="*40 + "\n")


# Quit the driver at the end
driver.quit()

Loading page: https://support.payever.org/hc/en-us/categories/360001870554-payever-Shop
Successfully loaded: https://support.payever.org/hc/en-us/categories/360001870554-payever-Shop
Loading additional page: https://support.payever.org/hc/en-us/articles/10030674417948-Set-up-the-shop
Successfully loaded: https://support.payever.org/hc/en-us/articles/10030674417948-Set-up-the-shop
It is available
URL: https://support.payever.org/hc/en-us/categories/360001870554-payever-Shop
Header: Sign in
H1: payever Shop
Sections:
  H2: Getting started
    UL: Set up the shop
    Links: ['https://support.payever.org/hc/en-us/articles/10030674417948-Set-up-the-shop']
      Additional URL: https://support.payever.org/hc/en-us/articles/10030674417948-Set-up-the-shop
      Additional H1: Set up the shop
      Paragraphs: ['', "How do you navigate in payever store. From the list of available applications select the store app. To beginn with you can automatically create the first store. Now let's customise 

# Extracting payever-Message page and it's link

In [53]:
# Initialize the driver
driver = init_driver()


# Specify the main URL to scrape
main_url = 'https://support.payever.org/hc/en-us/categories/4402737219730-payever-Message'

# Collect data from the main page
main_page_data = extract_data_from_page(main_url)

# Extract additional data from links within the UL elements
additional_data = []
for section in main_page_data['sections']:
    for ul in section['uls']:
        additional_urls = ul['links']
        for additional_url in additional_urls:
            additional_page_data = extract_simple_data_from_page(additional_url)
            additional_data.append(additional_page_data)



# Add additional data to the main page data
main_page_data['additional_data'] = additional_data

# Check if the file exists
if os.path.exists('scraped_data.json'):
    print('It is available')
    with open('scraped_data.json', 'r') as f:
        existing_data = json.load(f)
        
    # Extend the existing additional_data with the new additional_data
    if 'additional_data' in existing_data:
        existing_data['additional_data'].extend(additional_data)
    else:
        existing_data['additional_data'] = additional_data

    # Convert the dictionary to a JSON string
    json_string = json.dumps(existing_data, indent=4, separators=(',', ': '))

    # Insert two line breaks between JSON entries
    formatted_json_string = json_string.replace('\n', '\n\n')
    
    # Write the formatted JSON string back to the file
    with open('scraped_data.json', 'w') as f:
        f.write(formatted_json_string)
else:
    # Save data if file does not exist
    main_page_data['additional_data'] = additional_data
    with open('scraped_data.json', 'w') as f:
        json.dump(main_page_data, f, indent=4, separators=(',', ': '))

# Print the data
print(f"URL: {main_page_data['url']}")
print(f"Header: {main_page_data['header']}")
print(f"H1: {main_page_data['h1']}")
print("Sections:")
for section in main_page_data['sections']:
    print(f"  H2: {section['h2']}")
    for ul in section['uls']:
        print(f"    UL: {ul['text']}")
        print(f"    Links: {ul['links']}")
        for additional in additional_data:
            if additional['url'] in ul['links']:
                print(f"      Additional URL: {additional['url']}")
                print(f"      Additional H1: {additional['h1']}")
                print(f"      Paragraphs: {additional['paragraphs']}")
                print(f"      Images: {additional['images']}")
    print("\n" + "="*40 + "\n")


# Quit the driver at the end
driver.quit()

Loading page: https://support.payever.org/hc/en-us/categories/4402737219730-payever-Message
Successfully loaded: https://support.payever.org/hc/en-us/categories/4402737219730-payever-Message
Loading additional page: https://support.payever.org/hc/en-us/articles/4402745365266-Sell-via-WhatsApp
Successfully loaded: https://support.payever.org/hc/en-us/articles/4402745365266-Sell-via-WhatsApp
Loading additional page: https://support.payever.org/hc/en-us/articles/4402738677138-Sell-via-Facebook-Messenger
Successfully loaded: https://support.payever.org/hc/en-us/articles/4402738677138-Sell-via-Facebook-Messenger
Loading additional page: https://support.payever.org/hc/en-us/articles/4402783269650-Sell-via-Instagram-Messenger
Successfully loaded: https://support.payever.org/hc/en-us/articles/4402783269650-Sell-via-Instagram-Messenger
Loading additional page: https://support.payever.org/hc/en-us/articles/4418028695314-Additional-Terms-Facebook-WhatsApp-Business-Solution
Successfully loaded: ht

# Extracting payever-Shipping page and it's link

In [54]:
# Initialize the driver
driver = init_driver()

# Specify the main URL to scrape
main_url = 'https://support.payever.org/hc/en-us/categories/360002490060-payever-Shipping'

# Collect data from the main page
main_page_data = extract_data_from_page(main_url)

# Extract additional data from links within the UL elements
additional_data = []
for section in main_page_data['sections']:
    for ul in section['uls']:
        additional_urls = ul['links']
        for additional_url in additional_urls:
            additional_page_data = extract_simple_data_from_page(additional_url)
            additional_data.append(additional_page_data)



# Add additional data to the main page data
main_page_data['additional_data'] = additional_data

# Check if the file exists
if os.path.exists('scraped_data.json'):
    print('It is available')
    with open('scraped_data.json', 'r') as f:
        existing_data = json.load(f)
        
    # Extend the existing additional_data with the new additional_data
    if 'additional_data' in existing_data:
        existing_data['additional_data'].extend(additional_data)
    else:
        existing_data['additional_data'] = additional_data

    # Convert the dictionary to a JSON string
    json_string = json.dumps(existing_data, indent=4, separators=(',', ': '))

    # Insert two line breaks between JSON entries
    formatted_json_string = json_string.replace('\n', '\n\n')
    
    # Write the formatted JSON string back to the file
    with open('scraped_data.json', 'w') as f:
        f.write(formatted_json_string)
else:
    # Save data if file does not exist
    main_page_data['additional_data'] = additional_data
    with open('scraped_data.json', 'w') as f:
        json.dump(main_page_data, f, indent=4, separators=(',', ': '))

# Print the data
print(f"URL: {main_page_data['url']}")
print(f"Header: {main_page_data['header']}")
print(f"H1: {main_page_data['h1']}")
print("Sections:")
for section in main_page_data['sections']:
    print(f"  H2: {section['h2']}")
    for ul in section['uls']:
        print(f"    UL: {ul['text']}")
        print(f"    Links: {ul['links']}")
        for additional in additional_data:
            if additional['url'] in ul['links']:
                print(f"      Additional URL: {additional['url']}")
                print(f"      Additional H1: {additional['h1']}")
                print(f"      Paragraphs: {additional['paragraphs']}")
                print(f"      Images: {additional['images']}")
    print("\n" + "="*40 + "\n")


# Quit the driver at the end
driver.quit()

Loading page: https://support.payever.org/hc/en-us/categories/360002490060-payever-Shipping
Successfully loaded: https://support.payever.org/hc/en-us/categories/360002490060-payever-Shipping
Loading additional page: https://support.payever.org/hc/en-us/articles/360013519499-Set-up-payever-Shipping
Successfully loaded: https://support.payever.org/hc/en-us/articles/360013519499-Set-up-payever-Shipping
It is available
URL: https://support.payever.org/hc/en-us/categories/360002490060-payever-Shipping
Header: Sign in
H1: payever Shipping
Sections:
  H2: Getting Started
    UL: Set up payever Shipping
    Links: ['https://support.payever.org/hc/en-us/articles/360013519499-Set-up-payever-Shipping']
      Additional URL: https://support.payever.org/hc/en-us/articles/360013519499-Set-up-payever-Shipping
      Additional H1: Set up payever Shipping
      Paragraphs: ['', 'The shipping app makes the logistical part of your business much easier. You are able to automate processes and configure the

# Extracting payever-Transactions page and it's link

In [5]:
driver = init_driver()

main_url = 'https://support.payever.org/hc/en-us/categories/360002505639-payever-Transactions'

main_page_data = extract_data_from_page(main_url)


additional_data = []
for section in main_page_data['sections']:
    for ul in section['uls']:
        additional_urls = ul['links']
        for additional_url in additional_urls:
            additional_page_data = extract_simple_data_from_page(additional_url)
            additional_data.append(additional_page_data)



main_page_data['additional_data'] = additional_data


if os.path.exists('scraped_data.json'):
    print('It is available')
    with open('scraped_data.json', 'r') as f:
        existing_data = json.load(f)
        
    # Extend the existing additional_data with the new additional_data
    if 'additional_data' in existing_data:
        existing_data['additional_data'].extend(additional_data)
    else:
        existing_data['additional_data'] = additional_data

    # Convert the dictionary to a JSON string
    json_string = json.dumps(existing_data, indent=4, separators=(',', ': '))

    # Insert two line breaks between JSON entries
    formatted_json_string = json_string.replace('\n', '\n\n')
    
    # Write the formatted JSON string back to the file
    with open('scraped_data.json', 'w') as f:
        f.write(formatted_json_string)
else:
    # Save data if file does not exist
    main_page_data['additional_data'] = additional_data
    with open('scraped_data.json', 'w') as f:
        json.dump(main_page_data, f, indent=4, separators=(',', ': '))
# Print the data
print(f"URL: {main_page_data['url']}")
print(f"Header: {main_page_data['header']}")
print(f"H1: {main_page_data['h1']}")
print("Sections:")
for section in main_page_data['sections']:
    print(f"  H2: {section['h2']}")
    for ul in section['uls']:
        print(f"    UL: {ul['text']}")
        print(f"    Links: {ul['links']}")
        for additional in additional_data:
            if additional['url'] in ul['links']:
                print(f"      Additional URL: {additional['url']}")
                print(f"      Additional H1: {additional['h1']}")
                print(f"      Paragraphs: {additional['paragraphs']}")
                print(f"      Images: {additional['images']}")
    print("\n" + "="*40 + "\n")

# Quit the driver at the end
driver.quit()

Loading page: https://support.payever.org/hc/en-us/categories/360002505639-payever-Transactions
Successfully loaded: https://support.payever.org/hc/en-us/categories/360002505639-payever-Transactions
Loading additional page: https://support.payever.org/hc/en-us/articles/360013573960-Edit-transactions-table
Successfully loaded: https://support.payever.org/hc/en-us/articles/360013573960-Edit-transactions-table
Loading additional page: https://support.payever.org/hc/en-us/articles/360013644359-Filter-transactions
Successfully loaded: https://support.payever.org/hc/en-us/articles/360013644359-Filter-transactions
Loading additional page: https://support.payever.org/hc/en-us/articles/360013573700-Export-transactions
Successfully loaded: https://support.payever.org/hc/en-us/articles/360013573700-Export-transactions
It is available
URL: https://support.payever.org/hc/en-us/categories/360002505639-payever-Transactions
Header: Sign in
H1: payever Transactions
Sections:
  H2: Getting Started
    U